# 🤖 Module 5.4: Algorithmic Trading

**Time:** 8 hours | **Difficulty:** 🔴 Advanced

## Learning Objectives
- ✅ Trading strategy development
- ✅ Backtesting framework
- ✅ Performance metrics
- ✅ Risk management

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Generate Sample Data

In [ ]:
np.random.seed(42)
dates = pd.date_range('2020-01-01', periods=500, freq='B')
prices = 100 * np.exp(np.cumsum(np.random.normal(0.0003, 0.015, 500)))

df = pd.DataFrame({'Close': prices}, index=dates)
df['Returns'] = df['Close'].pct_change()
print(df.tail())

## 2. Moving Average Crossover Strategy

In [ ]:
# Calculate indicators
df['SMA_20'] = df['Close'].rolling(20).mean()
df['SMA_50'] = df['Close'].rolling(50).mean()

# Generate signals
df['Signal'] = 0
df.loc[df['SMA_20'] > df['SMA_50'], 'Signal'] = 1  # Long
df.loc[df['SMA_20'] < df['SMA_50'], 'Signal'] = -1  # Short

# Position changes
df['Position'] = df['Signal'].shift(1)  # Trade on next day

print(df[['Close', 'SMA_20', 'SMA_50', 'Signal']].tail(10))

## 3. Backtesting

In [ ]:
# Strategy returns
df['Strategy_Returns'] = df['Position'] * df['Returns']

# Cumulative returns
df['Buy_Hold'] = (1 + df['Returns']).cumprod()
df['Strategy'] = (1 + df['Strategy_Returns']).cumprod()

# Plot
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

axes[0].plot(df.index, df['Close'], label='Price')
axes[0].plot(df.index, df['SMA_20'], label='SMA 20', alpha=0.7)
axes[0].plot(df.index, df['SMA_50'], label='SMA 50', alpha=0.7)
axes[0].set_title('Price & Moving Averages')
axes[0].legend()

axes[1].plot(df.index, df['Buy_Hold'], label='Buy & Hold')
axes[1].plot(df.index, df['Strategy'], label='Strategy')
axes[1].set_title('Cumulative Returns')
axes[1].legend()

plt.tight_layout()
plt.show()

## 4. Performance Metrics

In [ ]:
def calculate_metrics(returns, rf=0.02):
    """Calculate key performance metrics."""
    returns = returns.dropna()
    
    # Annualized metrics
    ann_return = returns.mean() * 252
    ann_vol = returns.std() * np.sqrt(252)
    sharpe = (ann_return - rf) / ann_vol
    
    # Drawdown
    cum_returns = (1 + returns).cumprod()
    rolling_max = cum_returns.expanding().max()
    drawdown = (cum_returns - rolling_max) / rolling_max
    max_dd = drawdown.min()
    
    # Win rate
    win_rate = (returns > 0).sum() / len(returns)
    
    return {
        'Annual Return': f"{ann_return:.2%}",
        'Annual Volatility': f"{ann_vol:.2%}",
        'Sharpe Ratio': f"{sharpe:.2f}",
        'Max Drawdown': f"{max_dd:.2%}",
        'Win Rate': f"{win_rate:.2%}"
    }

print("Strategy Performance:")
for k, v in calculate_metrics(df['Strategy_Returns']).items():
    print(f"  {k}: {v}")

print("\nBuy & Hold Performance:")
for k, v in calculate_metrics(df['Returns']).items():
    print(f"  {k}: {v}")

## 5. Mean Reversion Strategy

In [ ]:
# Bollinger Bands
df['MA_20'] = df['Close'].rolling(20).mean()
df['Upper'] = df['MA_20'] + 2 * df['Close'].rolling(20).std()
df['Lower'] = df['MA_20'] - 2 * df['Close'].rolling(20).std()

# Mean reversion signals
df['MR_Signal'] = 0
df.loc[df['Close'] < df['Lower'], 'MR_Signal'] = 1   # Buy oversold
df.loc[df['Close'] > df['Upper'], 'MR_Signal'] = -1  # Sell overbought

df['MR_Position'] = df['MR_Signal'].shift(1)
df['MR_Returns'] = df['MR_Position'] * df['Returns']
df['MR_Cumulative'] = (1 + df['MR_Returns'].fillna(0)).cumprod()

print("Mean Reversion Performance:")
for k, v in calculate_metrics(df['MR_Returns']).items():
    print(f"  {k}: {v}")

## 📝 Exercise: RSI Strategy
Implement an RSI-based trading strategy

In [ ]:
# YOUR CODE HERE
# 1. Calculate RSI
# 2. Generate signals (buy when RSI < 30, sell when RSI > 70)
# 3. Backtest and calculate performance


---
**Congratulations!** You've completed the Machine Learning track! 🎉